In [ ]:
import streamlit as st
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from PIL import Image
import numpy as np
import os

# Load text model once
@st.cache_resource
def load_text_model():
    tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
    model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
    return tokenizer, model

tokenizer, model = load_text_model()

# Title
st.title("Employee Emotion and Mood Analysis")

# Sidebar options
option = st.sidebar.selectbox("Choose Module", [
    "Text Emotion Detection",
    "Image Emotion Detection",
    "Video Emotion Detection",
    "Task Recommendation",
    "Mood Tracking",
    "Stress Detection",
    "Team Mood Analytics"
])

# Helper Functions
def get_text_emotion(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    predicted_label = torch.argmax(probabilities).item()
    labels = model.config.id2label
    return labels[predicted_label]

def get_face_emotion(uploaded_file):
    image = Image.open(uploaded_file)
    img = np.array(image)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    result = DeepFace.analyze(img_path=img, actions=["emotion"], enforce_detection=False)
    return result[0]["dominant_emotion"]

def recommend_task(emotion):
    recommendations = {
        "happy": "Engage in creative work or brainstorming.",
        "sad": "Take a short break or do a relaxing task.",
        "angry": "Do individual focused work.",
        "neutral": "Proceed with daily tasks.",
        "fear": "Talk to a colleague or take a break.",
        "disgust": "Avoid group activities for now.",
        "surprise": "Engage in dynamic tasks."
    }
    return recommendations.get(emotion, "No recommendation.")

# Mood log file path
log_file = "mood_log.csv"

def log_mood(entry):
    df = pd.DataFrame([entry])
    file_exists = os.path.exists(log_file)
    df.to_csv(log_file, mode='a', header=not file_exists, index=False)

# ========== Modules ==========

# 1. Text Emotion Detection
if option == "Text Emotion Detection":
    st.subheader("Text Emotion Detection")
    text = st.text_area("Enter your message:")
    if st.button("Detect Emotion"):
        if text.strip():
            emotion = get_text_emotion(text)
            st.success(f"Predicted Emotion: {emotion}")
            log_mood({"datetime": datetime.now(), "type": "text", "emotion": emotion})
        else:
            st.warning("Please enter some text.")

# 2. Image Emotion Detection
elif option == "Image Emotion Detection":
    st.subheader("Image Emotion Detection")
    uploaded_file = st.file_uploader("Upload an Image", type=["jpg", "png"])
    if uploaded_file is not None:
        st.image(uploaded_file)
        if st.button("Analyze Emotion"):
            emotion = get_face_emotion(uploaded_file)
            st.success(f"Predicted Emotion: {emotion}")
            log_mood({"datetime": datetime.now(), "type": "image", "emotion": emotion})

# 3. Video Emotion Detection
elif option == "Video Emotion Detection":
    st.subheader("Video Emotion Detection")
    st.info("Please run this code in your terminal or script due to webcam limitations:")
    st.code("""
import cv2
from deepface import DeepFace

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    try:
        result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        cv2.putText(frame, result[0]['dominant_emotion'], (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    except:
        pass
    cv2.imshow("Emotion Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
""")

# 4. Task Recommendation
elif option == "Task Recommendation":
    st.subheader("Task Recommendation")
    mood = st.selectbox("Select your current mood", ["happy", "sad", "angry", "neutral", "fear", "disgust", "surprise"])
    task = recommend_task(mood)
    st.success(f"Recommended Task: {task}")

# 5. Mood Tracking
elif option == "Mood Tracking":
    st.subheader("Mood Tracking")
    if os.path.exists(log_file):
        df = pd.read_csv(log_file)
        st.dataframe(df.tail(10))
    else:
        st.info("No mood log data found.")

# 6. Stress Detection
elif option == "Stress Detection":
    st.subheader("Stress Detection")
    if os.path.exists(log_file):
        df = pd.read_csv(log_file)
        negative_emotions = ["sad", "angry", "fear", "disgust"]
        df["stress"] = df["emotion"].isin(negative_emotions)
        stress_count = df["stress"].sum()
        if stress_count > 5:
            st.error("High stress level detected! Take immediate action.")
        else:
            st.success("Stress levels are under control.")
    else:
        st.info("No mood log data found.")

# 7. Team Mood Analytics
elif option == "Team Mood Analytics":
    st.subheader("Team Mood Analytics")
    if os.path.exists(log_file):
        df = pd.read_csv(log_file)
        st.write("Mood Frequency")
        fig, ax = plt.subplots()
        sns.countplot(x="emotion", data=df, ax=ax)
        st.pyplot(fig)
    else:
        st.info("No mood log data found.")
